## КИНОПОИСК


Составить таблицу топ-100 фильмов с сайта https://www.kinopoisk.ru

В таблицу включить следующие поля:
    
    оригинальное название
    год выпуска
    страна (-ы)
    рейтинг Кинопоиска (в баллах)
    место в рейтинге Кинопоиска



In [1]:
import requests
import pandas as pd
import json
import numpy as np

Отправляю get запрос к сайту https://kinopoisk.dev/ предварительно зарегистрировавшись и получив ключ (token)
На основе документации к API формируем параметры для запроса.

sortField - поле по которому проводим фильтрацию. Выбираем значение rating.kp, так как нам надо отобрать 100 фильмов с наивысшим рейтингом kp

sortType - сортировку проводим в обратном порядке, от фильмов с наивысшим рейтингом kp к фильмам с наименьшим рейтингом.
Поэтому значение для этого параметра -1

page - полученные данные разместить на 1 странице

limit - ограничение в 100 фильмов

typeNumber - значение 1 - это movie, можно так же использовать параметр type, тогда значением будет "movie".
Есть значения для сериалов, для мультфильмов. Например, если указать значение параметра 3, то получим тор 100 для мультфильмов

isSeries - признак сериалов.

В headers указываем для значения "X-API-KEY" свой ключ API (token)


In [4]:
response = requests.get('https://api.kinopoisk.dev/v1.3/movie',
                        params={"sortField": "rating.kp",
                                "sortType": "-1",
                                "page": "1",
                                "limit": "100",
                                "typeNumber":1,
                                "isSeries":0},  
                        headers={"X-API-KEY":"XXXXXXX-XXXXXXX-XXXXXXX-XXXXXXX"}) # Ключ API

In [5]:
data = response.json()

В полученном  ответе на запрос отбираем  только колонку docs, так как именно в этой колонке находится нужная информация

In [6]:
data = data['docs'] 

Формируем датафрейм, выбирая нужные нам колонки

In [7]:
df = pd.DataFrame(data, columns=['name', 'alternativeName', 'year', 'countries', 'rating'])


Создаем колонку с местом в рейтинге на основе рейтинга ккинопоиска. Значения в этой колонке на 1 больше, чем индекс строки
так как индексация начинается с нуля.

In [8]:
df['rating_place'] = df.index+1

Оставляем в колонке rating только рейтинг кинопоиска, для этого используя lambda функцию берем значение ключа словаря 'kp'

In [9]:
df['rating'] = df['rating'].apply(lambda x: x['kp'])

Пишем функцию для преобразования  столбца countries в строку, так как мы получаем эту колонку со значениями в виде списка, содержащего словари, в которых ключом является 'name', а значением название страны. Затем проходясь в цикле формируем строку, в которой через запятые указаны страны, если их несколько. На выход передаем эту строку.

In [10]:
def concat_movies(sp):
    s = []
    sl =''
    sp1 = len(sp)
    for i in range(sp1):
        s.append(list(sp[i].values())[0])
    for i in s:
        sl = sl + ', ' + i
    return sl[2:]


Применяем написанную функцию к каждому элементу колонки countries

In [11]:
df['countries'] = df['countries'].apply(concat_movies)

Оригинальное название фильма - это название фильма, которое было ему дано при съемке в определенной стране. Исследование полей в полученном ответе с сайта показало, что это поле name для фильмов в колонке countries для которых значения соответствуют Россия, Украина, СССР. Для фильмов снятых в других странах оригинальное название находится в колонке alternativeName. В колонке name в этом случае возвращается значение None. Поэтому напишем функцию, которая будет присваивать значению name оригинальное название в зависимости от страны, где этот фильм был снят.Таким образом у нас не будет пропусков в колонке name, а названия для фильмов будут либо на русском, либо на aнглийском или допустим на французском языке, в звисимости от того, что написано в колонке alternativeName, если в колонке name значение None.

In [16]:
def origin_name(name, alternativeName, countries):
    if countries != 'Россия' and countries != 'СССР' and countries != 'Украина':
        return alternativeName
    else:
        return name

In [ ]:
Создаем колонку name_origin, применяя написанную функцию и передаем в нее нужные ннам колонки.

In [17]:
df['name_origin']= df[['name', 'alternativeName', 'countries']].apply(lambda df: origin_name(df['name'], df['alternativeName'], df['countries']), axis=1)

Можно произвести векторизацию нашей функции, тогда она будет работать быстрее как функции, которые создавались специально для pandas.

In [18]:
df['name_origin']= np.vectorize(origin_name)(df['name'], df['alternativeName'], df['countries'])

In [19]:
df['name'] = df['name_origin']

Удаляем колонку 'name_origin' так как мы уже перенесли названия фильмов в колонку name

In [20]:
df.drop(['name_origin'], axis = 1,inplace=True)

Оставляем в датафрейме нужные нам колонки

In [21]:
df = df[['name', 'year', 'countries', 'rating','rating_place']]

In [22]:
df.head(10) # выводим для примера 10 строк из получившейся таблицы.

,name,year,countries,rating,rating_place
0,Lords of the Lockerroom,1999,США,9.399,1
1,BTS: Blood Sweat & Tears,2016,Корея Южная,9.352,2
2,Hamilton's America,2016,США,9.245,3
3,Threat Level Midnight: The Movie,2011,"Великобритания, США",9.232,4
4,Жизнь человека. Последнее интервью,2020,Россия,9.180,5
5,Boosh Music,2005,Великобритания,9.169,6
6,Depeche Mode: Devotional,1993,Великобритания,9.166,7
7,Он вам не Димон,2017,Россия,9.127,8
8,The Shawshank Redemption,1994,США,9.108,9
9,The Green Mile,1999,США,9.074,10


In [23]:
df.head(100) # пять строк от начала и 5 строк от конца датафрейма.

,name,year,countries,rating,rating_place
0,Lords of the Lockerroom,1999,США,9.399,1
1,BTS: Blood Sweat & Tears,2016,Корея Южная,9.352,2
2,Hamilton's America,2016,США,9.245,3
3,Threat Level Midnight: The Movie,2011,"Великобритания, США",9.232,4
4,Жизнь человека. Последнее интервью,2020,Россия,9.180,5
...,...,...,...,...,...
95,Дальше — тишина,1978,СССР,8.729,96
96,Без срока давности. Пепел «Зимнего волшебства»,2017,Россия,8.728,97
97,Pink Floyd: P. U. L. S. E. Live at Earls Court,1994,Великобритания,8.726,98
98,Giuseppe Moscati: L'amore che guarisce,2007,Италия,8.726,99


По результатам выборки хочу обратить внимание на возможные расхождения при проверке данного списка на сайте кинопоиска
https://www.kinopoisk.ru/top/navigator/m_act[is_film]/on/order/rating/#results
Как выяснилось, фильтрация в предложенном API отличается от фильтрации пользователем на сайте. К нам в выборку фильмы попадают на основе рейтинга и параметра type, где для них указано значение movie.

Выводим получившийся датафрейм в таблицу Excel.

In [24]:
df.to_excel('top100_films.xlsx', index=False)